## Fine-tune for Multi-class Classification using OOB BERT MLM

#### Prerequisites 

In [1]:
%%capture 

!pip install sagemaker==2.100.0
!pip install jedi==0.17  # this is a requirement for pygmentize to work

#### Imports  

In [2]:
from sagemaker.huggingface import HuggingFace
from sagemaker import get_execution_role
from sagemaker import Session
import sagemaker
import logging

##### Setup logging

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies

In [4]:
logger.info(f'[Using SageMaker: {sagemaker.__version__}]')

[Using SageMaker: 2.100.0]


#### Essentials 

In [5]:
session = Session()
ROLE = get_execution_role()
S3_BUCKET = session.default_bucket()
ENTRY_POINT = 'fine_tune.py'
SOURCE_DIR = './src'
INSTANCE_TYPE = 'ml.p3.16xlarge'
INSTANCE_COUNT = 4
EBS_VOLUME_SIZE = 1024
TRANSFORMERS_VERSION = '4.17.0'
PYTORCH_VERSION = '1.10.2'
PYTHON_VERSION = 'py38'

In [6]:
logger.info(f'S3 bucket = {S3_BUCKET}')

S3 bucket = sagemaker-us-east-1-119174016168


#### View training script

In [7]:
!pygmentize ./src/fine_tune.py

from transformers import BertForSequenceClassification
from transformers import DataCollatorWithPadding
from sklearn.preprocessing import LabelEncoder
from transformers import TrainingArguments
from transformers import BertTokenizerFast
from datasets import load_dataset
from transformers import pipeline 
from transformers import Trainer
from datasets import DatasetDict
import pandas as pd
import transformers
import sagemaker
import argparse
import datasets
import logging
import random
import shutil
import torch
import boto3
import time
import sys
import os


# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.getLevelName('INFO'), 
                    handlers=[logging.StreamHandler(sys.stdout)], 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Log versions of dependencies
logger.info(f'[Using Transformers: {transformers.__version__}]')
logger.info(f'[Using SageMaker: {sagemaker.__version__}]')
logger.info(f'[Usin

#### Create the estimator 

* Documentation on SageMaker HuggingFace Estimator can be found [here](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html)

In [ ]:
DATA = {'train': f's3://{S3_BUCKET}/data'}

In [ ]:
MAX_LENGTH = 512  # Context size for BERT tokenizer 
CHUNK_SIZE = 128  
TRAIN_EPOCHS = 1
BATCH_SIZE = 32

In [ ]:
HYPERPARAMETERS = {'s3_bucket': S3_BUCKET, 
                   'max_len': MAX_LENGTH,
                   'chunk_size': CHUNK_SIZE,
                   'num_train_epochs': TRAIN_EPOCHS, 
                   'per_device_train_batch_size': BATCH_SIZE}

In [ ]:
DISTRIBUTION_STRATEGY = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

In [ ]:
huggingface_estimator = HuggingFace(entry_point=ENTRY_POINT, 
                                    source_dir=SOURCE_DIR, 
                                    role=ROLE, 
                                    instance_type=INSTANCE_TYPE, 
                                    instance_count=INSTANCE_COUNT,
                                    volume_size=EBS_VOLUME_SIZE,
                                    hyperparameters=HYPERPARAMETERS,
                                    distribution=DISTRIBUTION_STRATEGY,
                                    transformers_version=TRANSFORMERS_VERSION, 
                                    pytorch_version=PYTORCH_VERSION, 
                                    py_version=PYTHON_VERSION, 
                                    disable_profiler=True,
                                    debugger_hook_config=False)

In [ ]:
huggingface_estimator.fit(DATA, wait=False)